In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
from ngram_manager import NgramManager

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [3]:
class ClusterNgram:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        
        
    
    def cluster_n_grams(self, ngrams, num_cluster):
        X = self.__convert_ngrams_to_vectors(ngrams)
        kclusterer = KMeansClusterer(num_cluster, distance=nltk.cluster.util.cosine_distance, repeats=100)
        y = kclusterer.cluster(X, assign_clusters=True)
        
        top_indices = self.__find_top_n_gram_indices(X, y, kclusterer.means())
        
        return np.array(ngrams)[top_indices]
        
        
    
    def __find_top_n_gram_indices(self, X, y, centroids):
        min_dis = len(centroids)*[100]
        top_indices = len(centroids)*[-1]
        for i in range(0, len(centroids)):
            for j in range(0, len(X)):
                if y[j] != i:
                    continue
                
                dis = nltk.cluster.util.cosine_distance(X[j], centroids[i])
                if min_dis[i] > dis:
                    min_dis[i] = dis
                    top_indices[i] = j
        
        return top_indices
                    
    
    def __convert_ngrams_to_vectors(self, ngrams):
        X = []
        for ngram in ngrams:
            words = tokenize_uk.tokenize_words(ngram)
            x = word2vec[words[0]]
            for i in range(1, len(words)):
                x = x + word2vec[words[i]]
            X.append(x)
        return X

In [4]:
embeddings_file = "./embeddings/ubercorpus.cased.tokenized.word2vec.300d"

In [5]:
#word2vec = KeyedVectors.load_word2vec_format(embeddings_file)

In [6]:
#cluster_ngram = ClusterNgram(word2vec)

In [7]:
uk_stop_words = read_stop_words('./data/ukrainian-stopwords.txt')

In [8]:
pos_unigram_scores = pd.read_csv('./data/unigram-pmi-positive-scores.csv')
pos_bigram_scores = pd.read_csv('./data/bigram-pmi-positive-scores.csv')
pos_trigram_scores = pd.read_csv('./data/trigram-pmi-positive-scores.csv')

In [27]:
p1 = pos_unigram_scores.rename(columns={'ngram':'uni-gram'})
p2 = pos_bigram_scores.rename(columns={'ngram':'bi-gram'})
p3 = pos_trigram_scores.rename(columns={'ngram':'tri-gram'})

In [28]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [29]:
display(p1.head())
display(p2.head())
display(p3.head())

,uni-gram,score
0,торговий,0.251974
1,стильний,0.251648
2,просторо,0.250339
3,шикарне,0.249222
4,відмінне,0.249179


,bi-gram,score
0,місцезнаходження супер,0.255065
1,привітна господиня,0.255065
2,єр чистота,0.255065
3,чудове співвідношення,0.255065
4,просторі кімнати,0.255065


,tri-gram,score
0,персонал ввічливий готовий,0.255065
1,приємний персонал смачні,0.255065
2,приємний персонал хороший,0.255065
3,приємний персонал чистий,0.255065
4,персонал смачні ситні,0.255065


In [9]:
neg_unigram_scores = pd.read_csv('./data/unigram-pmi-negative-scores.csv')
neg_bigram_scores = pd.read_csv('./data/bigram-pmi-negative-scores.csv')
neg_trigram_scores = pd.read_csv('./data/trigram-pmi-negative-scores.csv')

In [30]:
n1 = neg_unigram_scores.rename(columns={'ngram':'uni-gram'})
n2 = neg_bigram_scores.rename(columns={'ngram':'bi-gram'})
n3 = neg_trigram_scores.rename(columns={'ngram':'tri-gram'})

In [31]:
display(n1.head())
display(n2.head())
display(n3.head())

,uni-gram,score
0,зламаний,0.347339
1,плямами,0.347069
2,слабка,0.347034
3,дверцята,0.344510
4,павутина,0.344362


,bi-gram,score
0,дивний запах,0.352442
1,жахлива звукоізоляція,0.352442
2,номері погана,0.352442
3,номері погано,0.352442
4,скрипучі ліжко,0.352442


,tri-gram,score
0,відсутність питної води,0.352442
1,запах ванній кімнаті,0.352442
2,звукоізоляція залишає бажати,0.352442
3,порожній міні бар,0.352442
4,слабкий сигнал wi,0.352442


In [10]:
pos_ngram_mng = NgramManager(unigram_scores=pos_unigram_scores, 
                             bigram_scores=pos_bigram_scores, 
                             trigram_scores=pos_trigram_scores,
                            stop_words=uk_stop_words,
                            threshold=0.1)

In [11]:
text =  "Готель дуже зручно розташований. Якщо вам в номері не сидіти то можна спокійно бронювати. Сніданок хороший. Номери стандартні НЕ великі. Персонал ввічливий. Чисто. Після ремонту свіжо. Проблема була з водою, гаряча швидко закінчилася. Але на наступний день з'явилася)). Тапки були одні речі. Мабуть комплектацією дійсно не заморочуються. Але якщо б сказали принесли напевно, ми не просили. Вид з нашого Номери 305 був на вікна житлового будинку, але якщо вийти на балкончик і подивитися на право красиво. Відмінний вид з ресторану на місто. Таксі нам викликали на прощання, доїхали до вокзалу за 3 ціни))))) але зате водій був ввічливий і машина хороша. На фото вид з ресторану."

In [12]:
text = "Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."

In [22]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [21]:
print("\n===Review text===")
print(text)
print("\n===Keywords===")
display(pos_ngram_mng.find_important_unigrams(text))
display(pos_ngram_mng.find_important_bigrams(text))
display(pos_ngram_mng.find_important_trigrams(text))


===Review text===
Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.
Щоденне прибирання.

===Keywords===


[('розташування', 0.23),
 ('хороше', 0.23),
 ('щоденне', 0.21),
 ('наявність', 0.17),
 ('парковка', 0.15)]

[('хороше розташування', 0.25),
 ('парковка території', 0.21),
 ('щоденне прибирання', 0.21)]

[('парковка території готелю', 0.17)]

In [15]:
pos_ngram_mng.find_important_unigrams(text)

[('розташування', 0.23),
 ('хороше', 0.23),
 ('щоденне', 0.21),
 ('наявність', 0.17),
 ('парковка', 0.15)]

In [ ]:
neg_ngram_mng = NgramManager(unigram_scores=neg_unigram_scores, 
                             bigram_scores=neg_bigram_scores, 
                             trigram_scores=neg_trigram_scores,
                            stop_words=uk_stop_words, 
                            threshold=0.1)

In [ ]:
neg_ngram_mng.find_important_bigrams(text)

In [ ]:
neg_ngram_mng.find_important_trigrams(text)

In [ ]:
pos_ngram_mng.find_important_unigrams(text)

In [ ]:
neg_ngram_mng.find_important_unigrams(text)

In [ ]:
import stanfordnlp

In [ ]:
stanfordnlp.download('uk')

In [ ]:
nlp = stanfordnlp.Pipeline(lang="uk")

In [ ]:
text = "Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."

In [ ]:
import pickle

In [ ]:
# open a file, where you stored the pickled data
file = open("pos_tfidf_transformer.pickle", 'rb')

# dump information to that file
pos_tfidf_transformer = pickle.load(file)

# close the file
file.close()

In [ ]:
# open a file, where you stored the pickled data
file = open("pos_cv.pickle", 'rb')

# dump information to that file
pos_cv = pickle.load(file)

# close the file
file.close()

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [ ]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
pos_cv.get_feature_names()

In [ ]:
# get the document that we want to extract keywords from
doc="Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."
#doc  = "Готель дуже зручно розташований. Якщо вам в номері не сидіти то можна спокійно бронювати. Сніданок хороший. Номери стандартні НЕ великі. Персонал ввічливий. Чисто. Після ремонту свіжо. Проблема була з водою, гаряча швидко закінчилася. Але на наступний день з'явилася)). Тапки були одні речі. Мабуть комплектацією дійсно не заморочуються. Але якщо б сказали принесли напевно, ми не просили. Вид з нашого Номери 305 був на вікна житлового будинку, але якщо вийти на балкончик і подивитися на право красиво. Відмінний вид з ресторану на місто. Таксі нам викликали на прощання, доїхали до вокзалу за 3 ціни))))) але зате водій був ввічливий і машина хороша. На фото вид з ресторану."

#generate tf-idf for the given document
pos_tf_idf_vector=pos_tfidf_transformer.transform(pos_cv.transform([doc, doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(pos_tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(pos_cv.get_feature_names(),sorted_items,10)

# now print the results
print("\n===Review text===")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])

In [ ]:
pos_cv.get_feature_names()[-1]

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [ ]:
display(pos_cv.get_feature_names()[-1])
print('a')